In [1]:
import torch
import time
import numpy as np
import pandas as pd
import csv
import torch.nn as nn
from sklearn.metrics import accuracy_score
from transformers import BertModel
from sklearn import metrics
from transformers import BertTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding,AutoModel
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os
os.environ['HTTP_PROXY'] = 'http://172.18.19.153:10809'
os.environ['HTTPS_PROXY'] = 'http://172.18.19.153:10809'
torch.cuda.set_device(0)

In [2]:
def before():
    #这是数据预处理部分
    data_train = pd.read_csv('Train.csv',encoding='GB18030')
    data_tmp=pd.DataFrame(data_train,columns=['ID', 'Text', 'Labels'])
    genres=[]
 
    for row in data_tmp.itertuples():
        str1=str(getattr(row, 'Labels'))
        str1=str1.strip('[]')
        temp=(str1.split(','))
        # 进行去重合并，结果为所有特征值
        for i in temp:
            i=i.strip()
            #存在影片没有分类数据，这种直接忽略掉
            if i=='(no genres listed)':
                continue
            #正常的影片分类数据加入到分类列表里面
            if i not in genres:
                genres.append(i)
    #对列进行扩充，增加了很多列    
    print(genres)
    num=pd.DataFrame(columns=genres)
    #num.index=1
    num.loc[0] = [0,0,0,0,0,0,0,0]
    data_train=pd.concat([data_train,pd.DataFrame(columns=genres)])
    # dTest=pd.concat([dTest,pd.DataFrame(columns=genres)])
    #对于新增加的列默认值设为0
    data_train.iloc[0:,3:]=0
    # dmovies.iloc[0:,4:]=0

    for index,row in data_tmp.iterrows():
        str1=str(getattr(row, 'Labels'))
        str1=str1.strip('[]')
        temp=(str1.split(','))
        for i in temp:
            i=i.strip()
            if i=='(no genres listed)':
                continue
            data_train.loc[index,i]=1
            num.loc[0,i]+=1
    # dTest=dTest.head(100)
    train=data_train.iloc[1000:]
    dev=data_train.iloc[:1000]
    train.drop(['ID','Labels'],axis=1,inplace=True)
    train.to_csv("data/train1.csv", index=False)
    dev.drop(['ID','Labels'],axis=1,inplace=True)
    dev.to_csv("data/test.csv", index=False)
    print(num)
before()      

["'Love'", "'Joy'", "'Anxiety'", "'Sorrow'", "'Expect'", "'Hate'", "'Surprise'", "'Anger'"]
   'Love'  'Joy'  'Anxiety'  'Sorrow'  'Expect'  'Hate'  'Surprise'  'Anger'
0   10225   5080       7877      6760      3695    2849         902     1717


/tmp/ipykernel_49918/1749584771.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['ID','Labels'],axis=1,inplace=True)
/tmp/ipykernel_49918/1749584771.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev.drop(['ID','Labels'],axis=1,inplace=True)


In [3]:
#录入数据
my_dataset = load_dataset('csv', data_files={'train':'data/train1.csv',
                                                'validation':'data/test.csv'})

#将数据拆分成训练集和测试集

# train_data.to_csv('test/train.csv',index=False)
# test_data.to_csv('test/test.csv',index=False)
print(my_dataset)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text', "'Love'", "'Joy'", "'Anxiety'", "'Sorrow'", "'Expect'", "'Hate'", "'Surprise'", "'Anger'"],
        num_rows: 25000
    })
    validation: Dataset({
        features: ['Text', "'Love'", "'Joy'", "'Anxiety'", "'Sorrow'", "'Expect'", "'Hate'", "'Surprise'", "'Anger'"],
        num_rows: 1000
    })
})


In [4]:
df_train=pd.DataFrame(my_dataset['train'])
df_dev=pd.DataFrame(my_dataset['validation'])

In [5]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 100
VALID_BATCH_SIZE = 100
EPOCHS = 15
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

In [6]:
target_cols = [col for col in df_train.columns if col not in ['Text', 'ID']]
target_cols

["'Love'",
 "'Joy'",
 "'Anxiety'",
 "'Sorrow'",
 "'Expect'",
 "'Hate'",
 "'Surprise'",
 "'Anger'"]

In [7]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [8]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(df_dev, tokenizer, MAX_LEN)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [10]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('bert-base-chinese')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,8)
        self.Sigmoid=torch.nn.Sigmoid()
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
       # print(features.shape)
        output = self.fc(features)
        #print(features)
        return output

model = BERTClass()
device='cuda'
model.to('cuda')
model.cuda(0)


BERTClass(
  (roberta): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [11]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
       
        outputs = model(ids, mask, token_type_ids)
        #print(outputs.shape)
        loss = loss_fn(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
for epoch in range(EPOCHS):
    train(epoch)

/data/zhangxiaoming/.conda/envs/env/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss:  0.7137538194656372
Epoch: 1, Loss:  0.3130794167518616
Epoch: 2, Loss:  0.272320032119751
Epoch: 3, Loss:  0.2516605257987976
Epoch: 4, Loss:  0.22794584929943085
Epoch: 5, Loss:  0.19077570736408234
Epoch: 6, Loss:  0.1667911559343338
Epoch: 7, Loss:  0.1535734087228775
Epoch: 8, Loss:  0.14284561574459076
Epoch: 9, Loss:  0.1298827975988388
Epoch: 10, Loss:  0.1349613219499588
Epoch: 11, Loss:  0.10123017430305481
Epoch: 12, Loss:  0.08653093129396439
Epoch: 13, Loss:  0.08087578415870667
Epoch: 14, Loss:  0.06292331963777542


In [12]:
#保存
torch.save(model, 'model.pt')
#读取
model = torch.load('model.pt')

In [17]:
num=0
all=0.0
df1=pd.read_csv("data/test.csv")
data_tmp=pd.DataFrame(df1,columns=['ID', 'Text', 'Labels'])
 
for ii in range(len(target_cols)):
        target_cols[ii]=target_cols[ii].replace("'","")
        target_cols[ii]=target_cols[ii].replace("'","")

print(target_cols) 
for _,data in enumerate(valid_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        
        outputs = model(ids, mask, token_type_ids)
        #print(outputs.shape)
        outputs=model.Sigmoid(outputs)
        #print(outputs.shape)
        outputs=outputs.cpu().detach().numpy()
        targets=targets.cpu().detach().numpy()
        #outputs=np.where(outputs>0.5,1,0)
        #print(outputs.shape[0])
       # print(type(targets[0][0]))
        # print( outputs[0])
        # print(targets[0])
        # print( outputs[1])
        # print(targets[1])
      

        for ii in range(outputs.shape[0]):
                pred=""
                if (outputs[ii]<0.5).all():
                        max1=np.max(outputs[ii])
                        for iii in range(len(outputs[ii])):
                                if(outputs[ii][iii]==max1):
                                        outputs[ii][iii]=1
                                else :
                                        outputs[ii][iii]=0
                else:
                        outputs=np.where(outputs>0.5,1,0)
                for iii in range(len(outputs[ii])):
                       if(outputs[ii][iii]==1):
                                pred=pred+target_cols[iii]+','
                pred=pred.rstrip(',')
        #         temp1=np.concatenate((ids[ii], mask[ii]), axis=0)
        #         temp2=np.concatenate((temp1, token_type_ids[ii]), axis=0)
        #         #print(temp2.shape)
        #         X_test.append(temp2)
        #         y_test.append(binary_array_to_number(targets[ii]))
        #        # print(ids[ii])
        #         my_string1 = np.array2string(ids[ii], separator=',')
        #         my_string2 = np.array2string(mask[ii], separator=',')
        #         my_string3 = np.array2string(token_type_ids[ii], separator=',')
        #         print(my_string1)
                #print(pred)
                df1.loc[num,'pred']=pred   
                # df1.loc[num,'ids']=my_string1      
                # df1.loc[num,'mask']=my_string2
                # df1.loc[num,'token_type_ids']=my_string3             
                accuracy = accuracy_score(targets[ii], outputs[ii])
                all+=accuracy
                num+=1

print(all/num)    
# df1.head(2)
df1.to_csv("pre_bert_chinese.csv",index=False)

        



['Love', 'Joy', 'Anxiety', 'Sorrow', 'Expect', 'Hate', 'Surprise', 'Anger']
0.8045


In [18]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets
outputs, targets = validation()
outputs = np.array(outputs) >= 0.3
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.238
F1 Score (Micro) = 0.5522580645161291
F1 Score (Macro) = 0.5051158200162302
